<a href="https://colab.research.google.com/github/Angelique28/Designing-Protein-Binding-Peptides---CECAM-Workshop/blob/main/notebooks/2_Target_Feasibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Workshop Notebook 2: Protein Setup & Visualization (15 min)**
In this notebook, we will:
1. Run AF2Bind to evaluate whether MDM2 has a feasible peptide-binding pocket.
2. Identify hotspot residues predicted to be critical for binding.
3. Visualize and annotate these hotspots on the protein structure.

### ⚠ IMPORTANT:
> We are re-using the MDM2 structure predicted in Notebook 1 (Boltz-2).  
If you skipped Notebook 1, please run it first.

### AF2Bind

AF2Bind is a simple and fast notebook that runs inference on the output obtained from [AlphaFold2](https://github.com/deepmind/alphafold).

We can use this to assess the feasibility of binding our target protein, and see where the designed peptides might bind.

For more details see preprint:

**AF2BIND: Predicting ligand-binding sites using the pair representation of AlphaFold2**
* Artem Gazizov, Anna Lian, Casper Alexander Goverde, Sergey Ovchinnikov, Nicholas F. Polizzi
* https://doi.org/10.1101/2023.10.15.562410

In [1]:
%%time
#@title **Set up our environment (~2 mins)**
#@markdown Please execute this cell by pressing the *Play* button on
#@markdown the left.
import os, time
from google.colab import drive
import subprocess
import torch

if not torch.cuda.is_available():
    print("⚠️ Warning: GPU runtime not detected. Please go to Runtime > Change runtime type > select GPU.")
else:
    print("✅ GPU detected:", torch.cuda.get_device_name(0))

if not os.path.isdir("params"):
  # get code
  print("installing ColabDesign")
  os.system("(mkdir params; apt-get install aria2 -qq; \
  aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar; \
  mkdir af2bind_params; \
  wget -qnc https://github.com/sokrypton/af2bind/raw/main/attempt_7_2k_lam0-03.zip; unzip attempt_7_2k_lam0-03.zip -d af2bind_params; \
  tar -xf alphafold_params_2021-07-14.tar -C params; touch params/done.txt )&")

  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")

  # download params
  if not os.path.isfile("params/done.txt"):
    print("downloading params")
    while not os.path.isfile("params/done.txt"):
      time.sleep(5)

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np

from colabdesign.af.alphafold.common import residue_constants
import pandas as pd
from google.colab import data_table
data_table._DEFAULT_FORMATTERS[float] = lambda x: f"{x:.3f}"
from IPython.display import display, HTML
import jax, pickle
import jax.numpy as jnp
from scipy.special import expit as sigmoid
import plotly.express as px

import py3Dmol
import matplotlib.pyplot as plt
from scipy.special import softmax
import copy
from colabdesign.shared.protein import renum_pdb_str
from colabdesign.af.alphafold.common import protein
from pathlib import Path

aa_order = {v:k for k,v in residue_constants.restype_order.items()}

def af2bind(outputs, mask_sidechains=True, seed=0):
  pair_A = outputs["representations"]["pair"][:-20,-20:]
  pair_B = outputs["representations"]["pair"][-20:,:-20].swapaxes(0,1)
  pair_A = pair_A.reshape(pair_A.shape[0],-1)
  pair_B = pair_B.reshape(pair_B.shape[0],-1)
  x = np.concatenate([pair_A,pair_B],-1)

  # get params
  if mask_sidechains:
    model_type = f"split_nosc_pair_A_split_nosc_pair_B_{seed}"
  else:
    model_type = f"split_pair_A_split_pair_B_{seed}"
  with open(f"af2bind_params/attempt_7_2k_lam0-03/{model_type}.pickle","rb") as handle:
    params_ = pickle.load(handle)
  params_ = dict(**params_["~"], **params_["linear"])
  p = jax.tree_map(lambda x:np.asarray(x), params_)

  # get predictions
  x = (x - p["mean"]) / p["std"]
  x = (x * p["w"][:,0]) + (p["b"] / x.shape[-1])
  p_bind_aa = x.reshape(x.shape[0],2,20,-1).sum((1,3))
  p_bind = sigmoid(p_bind_aa.sum(-1))
  return {"p_bind":p_bind, "p_bind_aa":p_bind_aa}

✅ GPU detected: Tesla T4
installing ColabDesign
downloading params
CPU times: user 4.91 s, sys: 728 ms, total: 5.64 s
Wall time: 1min 26s


In [2]:
#@title **Set up our Paths and mount Google Drive folder**

#@markdown We will set a project ID so that we can keep separate executions separated, and a step ID so that we can keep the outputs of each step separate
PROJECT_ID = "MDM2" #@param {type:"string"}
STEP_ID = "2"

#@markdown We will use Google Drive mounts for persistence between multiple notebooks in this tutorial.

#@markdown Log in with your Google account and give permissions to access the drive.
WORKSHOP_DIRECTORY = Path('/content/drive/MyDrive/cecam_workshop_2025_generative')
drive.mount(str(WORKSHOP_DIRECTORY.parent.parent))
STEP_PATH = WORKSHOP_DIRECTORY / 'projects' / PROJECT_ID / STEP_ID
STEP_PATH.mkdir(exist_ok = True, parents = True)

Mounted at /content/drive


In [3]:
#@title **Run AF2BIND** 🔬
#@markdown AF2Bind predicts binding feasibility for peptides and highlights *hotspot residues* that are likely to form strong interactions.

model_number = "0" #@param {type:"string"}
#@markdown AF2Bind provides binding scores for each residue.

#@markdown Higher scores suggest a stronger likelihood of being part of a peptide-binding site.

#@markdown > Look for clusters of high-scoring residues. These clusters define the *binding pocket* where we will target peptide design in the next notebook.

mask_sidechains = True
mask_sequence = False

pdb_filename = WORKSHOP_DIRECTORY / 'projects' / PROJECT_ID / '1' / 'boltz_results_target' / 'predictions' / 'target' / f'target_model_{model_number}.fixed.pdb'
target_chain = "A"

clear_mem()
af_model = mk_afdesign_model(protocol="binder", debug=True)
af_model.prep_inputs(pdb_filename=str(pdb_filename),
                     chain=target_chain,
                     binder_len=20,
                     rm_target_sc=mask_sidechains,
                     rm_target_seq=mask_sequence)

# split
r_idx = af_model._inputs["residue_index"][-20] + (1 + np.arange(20)) * 50
af_model._inputs["residue_index"][-20:] = r_idx.flatten()

af_model.set_seq("ACDEFGHIKLMNPQRSTVWY")
af_model.predict(verbose=False)

o = af2bind(af_model.aux["debug"]["outputs"],
            mask_sidechains=mask_sidechains)
pred_bind = o["p_bind"].copy()
pred_bind_aa = o["p_bind_aa"].copy()

#######################################################
labels = ["chain","resi","resn","p(bind)"]
data = []
for i in range(af_model._target_len):
  c = af_model._pdb["idx"]["chain"][i]
  r = af_model._pdb["idx"]["residue"][i]
  a = aa_order.get(af_model._pdb["batch"]["aatype"][i],"X")
  p = pred_bind[i]
  data.append([c,r,a,p])

df = pd.DataFrame(data, columns=labels)
data_table.enable_dataframe_formatter()
df_sorted = df.sort_values("p(bind)",ascending=False, ignore_index=True).rename_axis('rank').reset_index()
df_sorted.to_csv(STEP_PATH / f'af2bind_results_model_{model_number}.csv', index = False)
display(data_table.DataTable(df_sorted, min_width=100, num_rows_per_page=25, include_index=False))

<timed exec>:71: DeprecationWarning: Pickled array contains an aval with a named_shape attribute. This is deprecated and the code path supporting such avals will be removed. Please re-pickle the array.
<timed exec>:73: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).


,rank,chain,resi,resn,p(bind)
0,0,A,68,V,0.990510
1,1,A,29,L,0.955168
2,2,A,36,I,0.895178
3,3,A,50,V,0.880046
4,4,A,74,I,0.796151
...,...,...,...,...,...
78,78,A,7,P,0.016216
79,79,A,22,T,0.012767
80,80,A,14,K,0.011871
81,81,A,4,R,0.009973


from matplotlib import pyplot as plt
_df_0['rank'].plot(kind='hist', bins=20, title='rank')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['resi'].plot(kind='hist', bins=20, title='resi')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['p(bind)'].plot(kind='hist', bins=20, title='p(bind)')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['rank']
  ys = series['p(bind)']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_6.sort_values('rank', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('rank')
_ = plt.ylabel('p(bind)')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['rank']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'rank'}, axis=1)
              .sort_values('rank', ascending=True))
  xs = counted['rank']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_7.sort_values('rank', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('rank')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_8['rank'].plot(kind='line', figsize=(8, 4), title='rank')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['resi'].plot(kind='line', figsize=(8, 4), title='resi')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['p(bind)'].plot(kind='line', figsize=(8, 4), title='p(bind)')
plt.gca().spines[['top', 'right']].set_visible(False)

In [4]:
#@title Display Structure {run: "auto"}
#@markdown We can highlight the `TOP_N` predicted hotspot residues directly in 3D to see where binding is most likely to occur.

#@markdown > Be sure to note the hotspot residues that you're selecting, you'll need them for the next notebook!
import py3Dmol
from string import ascii_uppercase, ascii_lowercase

alphabet_list = list(ascii_uppercase+ascii_lowercase)

TOP_N = 6 #@param {type:"slider", min:0, max:20, step:1}

def interpolate_color(value: float) -> str:
    """
    Maps a float in [0.0, 1.0] to a color hex string based on defined stops.
    """

    # Clamp input to [0, 1]
    value = max(0.0, min(1.0, value))

    # Define stops
    stops = [
        (0.0, (255, 0, 0)),       # #FF0000
        (0.25, (255, 128, 128)),  # #FF8080
        (0.5, (255, 255, 255)),   # #FFFFFF
        (0.75, (128, 128, 255)),  # #8080FF
        (1.0, (0, 0, 255))        # #0000FF
    ]

    # Find the two stops to interpolate between
    for i in range(len(stops) - 1):
        (v1, c1), (v2, c2) = stops[i], stops[i + 1]
        if v1 <= value <= v2:
            # Normalized interpolation factor
            t = (value - v1) / (v2 - v1)
            # Linear interpolation per channel
            r = round(c1[0] + (c2[0] - c1[0]) * t)
            g = round(c1[1] + (c2[1] - c1[1]) * t)
            b = round(c1[2] + (c2[2] - c1[2]) * t)
            return f"#{r:02X}{g:02X}{b:02X}"

    # fallback (should never happen due to clamping)
    return "#000000"


def show_pdb(pdb_str,
             get_hotspots = True,
             chains=1, vmin=50, vmax=90,
             size=(800,480), hbondCutoff=4.0,
             Ls=None,
             animate=False):

  structure_format = 'pdb'

  view = py3Dmol.view(js='https://3Dmol.org/build/3Dmol-min.js', width=size[0], height=size[1])
  if animate:
    view.addModelsAsFrames(pdb_str, structure_format,{'hbondCutoff':hbondCutoff})
  else:
    view.addModel(pdb_str, structure_format) #, {'hbondCutoff':hbondCutoff})

  color_scheme = {'prop':'b','gradient': 'rwb','min':0,'max':100}
  hotspots = []
  if get_hotspots:
    for i, row in df_sorted.iterrows():
      view.setStyle(
          {'chain': row['chain'], 'resi': row['resi']},
          {'cartoon': {'color': interpolate_color(row['p(bind)'])}})

      if i < TOP_N:
        view.addStyle({'and':[{'chain':row['chain']},{'resi':row['resi']},{'resn':["GLY","PRO"],'invert':True},{'atom':['C','O','N'],'invert':True}]},
                      {'stick':{'colorscheme':color_scheme,'radius':0.3}})
        view.addStyle({'and':[{'chain':row['chain']},{'resi':row['resi']},{'resn':"GLY"},{'atom':'CA'}]},
                      {'sphere':{'colorscheme':color_scheme,'radius':0.3}})
        view.addStyle({'and':[{'chain':row['chain']},{'resi':row['resi']},{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                      {'stick':{'colorscheme':color_scheme,'radius':0.3}})
        hotspots.append(row['resi'])
  else:
    view.setStyle({'chain':'A'},{'cartoon': {'color':"#33ff33",}})
    view.setStyle({'chain':'B'},{'cartoon': {'color':"#FF0000",}})

  view.zoomTo()

  view.setHoverable(
    {},
    True,
    '''function(atom,viewer,event,container) {
        if(!atom.label) {
        atom.label = viewer.addLabel(atom.resn + atom.resi,{position: atom, backgroundColor: 'mintcream', fontColor:'black'});
        }}''',
    '''function(atom,viewer) {
        if(atom.label) {
        viewer.removeLabel(atom.label);
        delete atom.label;
        }
    }''',
    viewer=(0, 1)
  )

  if animate: view.animate()
  return view, hotspots

with (pdb_filename).open() as f:
  pdb_str = f.read()
view, hotspots = show_pdb(pdb_str)
view.show()
print('Selected hotspot residues for peptide design:', ','.join([str(i) for i in hotspots]))

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Selected hotspot residues for peptide design: 68,29,36,50,74,33


In [5]:
#@title Display Structure of experimental PDB for Comparison {run: "auto"}
#@markdown Compare the highlighted AF2Bind region above with the known p53-binding pocket.
import requests

PDB = '1YCR' #@param {type:"string"}
pdb_str = requests.get(f'https://files.rcsb.org/download/{PDB}.pdb').text

view, hotspots = show_pdb(pdb_str, get_hotspots = False, chains=2)
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Notebook Summary
- AF2Bind confirmed that MDM2 has a peptide-binding pocket.
- We identified and visualized hotspot residues critical for peptide binding.
- We now have a focused substructure with defined hotspot residues for guiding peptide generation.

➡️ Next: In Notebook 3, we will generate peptide candidates with RFDiffusion, ProteinMPNN, and ESM-IF1.
